In [ ]:
#!pip install hyperopt

In [ ]:
#!pip install eli5

     |████████████████████████████████| 106 kB 5.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np

import xgboost as xgb

from hyperopt import hp, fmin, tpe, STATUS_OK

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5

from eli5.sklearn import PermutationImportance

In [ ]:
cd "/content/drive/MyDrive/Colab Notebooks/dw_matrix/Matrix2/dw_matrix_car"

/content/drive/MyDrive/Colab Notebooks/dw_matrix/Matrix2/dw_matrix_car


In [ ]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [ ]:
SUFFIX_CAT = '_cat'

for feat in df.columns:

  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
cat_feats = [ x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [ x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [ ]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

In [ ]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

In [ ]:
#df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split (' ')[0] ))

df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else str(int(x.split ('cm')[0] .replace(' ',''))))

feats = ['param_napęd_cat', 'param_rok-produkcji', 'param_stan_cat', 'param_skrzynia-biegów_cat', 'param_faktura-vat_cat','param_moc','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa','seller_name_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat','feature_czujniki-parkowania-przednie_cat','feature_łopatki-zmiany-biegów_cat','feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

## Hyperopt

In [ ]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return { 'loss': np.abs(mean_mae), 'status': STATUS_OK }

  #space

xgb_reg_params = {
      
    'learning_rate':                hp.choice('learning_rate',              np.arange(0.05, 0.31, 0.05)),
    'max_depth':                    hp.choice('max_depth',                  np.arange(5, 16, 1, dtype=int)),
    'subsample':                    hp.quniform('subsample', 0.5, 1, 0.01),
    'colsample_bytree':             hp.quniform('colsample_bytree', 0.5, 1, 0.01),
    'objective':                    'reg:squarederror', 
    'n_estimators': 100,
    'seed': 0

  }

  #run

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.76, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.81}
Training with params: 
{'colsample_bytree': 0.8300000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8200000000000001}
Training with params: 
{'colsample_bytree': 0.98, 'learning_rate': 0.2, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.73}
Training with params: 
{'colsample_bytree': 0.74, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.68}
Training with params: 
{'colsample_bytree': 0.68, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 0.61, 'learning_rate': 0.15000000000000002, 'max_d

{'colsample_bytree': 0.92,
 'learning_rate': 2,
 'max_depth': 8,
 'subsample': 0.9500000000000001}